## Network X functions

In [112]:
## Import libraries
import osmnx as ox
import networkx as nx
import math
import dash_leaflet as dl
from jupyter_dash import JupyterDash
from dash import html
import dash

Test a Tolworth 04 route

In [84]:
start_node = 23779844
G_tolworth = ox.graph_from_point((51.3829463, -0.2933327), dist=1000, network_type='drive')  # Adjust lat/lon to roughly London
node_id = 10282513496
centre_lat = 51.3829463
centre_lon = -0.2933327

In [3]:
type(G_tolworth)

networkx.classes.multidigraph.MultiDiGraph

In [90]:
# function to get nodes on a road
def get_nodes_on_road(G, road_name: str) -> set:
    """_summary_
    Given a collection of nodes via a networkx.classes.multidigraph.MultiDiGraph find
    all of the nodes on a given road

    Args:
        G (OSMNX digraph): Collection of nodes to investigate
        road_name (str): Road name 

    Returns:
        set: set of all osmnx nodes on a road
    """
    nodes = set()
    for u, v, data in G.edges(data=True): # u is the start node, v is the edge node, data is edge data
        if data.get('name') == road_name:
            nodes.add(u)
            nodes.add(v)
    return nodes

# Function to get the angle of turn nodes
def turn_angle(current_node: dict, shared_node: dict, neighbour_node: dict):
    """_summary_

    Args:
        current_node (dict): _description_
        shared_node (dict): _description_
        neighbour_node (dict): _description_

    Returns:
        _type_: _description_
    """
    x1 =  shared_node['x'] - current_node['x']
    y1 =  shared_node['y'] - current_node['y']
    x2 =  neighbour_node['x'] - shared_node['x']
    y2 =  neighbour_node['y'] - shared_node['y']
    
    # print(f'x1: {x1}, y1: {y1}' )
    # print(f'x2: {x2}, y2: {y2}' )

    angle1 = math.atan2(y1, x1)
    angle2 = math.atan2(y2, x2)
    
    # Signed angle difference
    angle_deg = math.degrees(angle1 - angle2)

    return angle_deg + 180

# Turn direction 
def get_turn_node(G, current_road_name: str, next_road_name: str, current_node: str, direction: str):
    """_summary_

    Args:
        G (_type_): _description_
        current_road_name (str): _description_
        next_road_name (str): _description_
        current_node (str): _description_
        direction (str): _description
    """
    # 1) Get the nodes on the current road
    curr_road_nodes = get_nodes_on_road(G, current_road_name)
    # 2) Get the nodes on the next road
    next_road_nodes = get_nodes_on_road(G, next_road_name)
    # 3) Find the nodes common to both to get junction nodes 
    junc_node = list(curr_road_nodes.intersection(next_road_nodes))[0] # will need to deal with multiple nodes
    # 4) Get neighbours of common node
    junc_node_neighbours = [neighbour for neighbour in G.neighbors(junc_node) if neighbour in next_road_nodes]
    # 5) Get angles of the turns
    neigbour_angles = [turn_angle(G.nodes[current_node], G.nodes[junc_node], G.nodes[x]) for x in junc_node_neighbours]
    # 7) Select Neighbour node dependent on turn direction
    if direction == 'left':
        final_node = junc_node_neighbours[neigbour_angles.index(min(neigbour_angles))]
    elif direction == "right":
        final_node = junc_node_neighbours[neigbour_angles.index(max(neigbour_angles))]
    else:
        pass # may need to add this??
    # 7) Get the route from current node to final node
    current_node_to_junction_pth = nx.shortest_path(G, source=current_node, target=junc_node)
    junction_to_turn_pth = nx.shortest_path(G, junc_node, final_node)
    
    return current_node_to_junction_pth[:-1] + junction_to_turn_pth
      
# test functions section of cell ======================================================= ##
#get_turn_candiates(G_tolworth, 'Douglas Road')

# turn_angle(G_tolworth.nodes[start_node], G_tolworth.nodes[2578590287], G_tolworth.nodes[304437]), turn_angle(G_tolworth.nodes[start_node], G_tolworth.nodes[2578590287], G_tolworth.nodes[304438])
# turn_angle(G_tolworth.nodes[23780711], G_tolworth.nodes[start_node], G_tolworth.nodes[2578590287])
get_turn_node(G_tolworth, 'Douglas Road', 'Ewell Road', 23780711, 'left'), get_turn_node(G_tolworth, 'Douglas Road', 'Ewell Road', 23780711, 'right'),get_turn_node(G_tolworth, 'Douglas Road', 'Ewell Road', 2578590287, 'left')

([23780711, 23779844, 2578590287, 304437],
 [23780711, 23779844, 2578590287, 304438],
 [2578590287, 304437])

In [57]:
#G_tolworth.nodes[start_node]['x'] - G_tolworth.nodes[2578590287]['x']
[x for x in G_tolworth.neighbors(2578590287)]
G_tolworth.nodes[start_node], G_tolworth.nodes[2578590287], G_tolworth.nodes[304437]

({'y': 51.381894, 'x': -0.2938243, 'street_count': 3},
 {'y': 51.3844335, 'x': -0.2926397, 'street_count': 3},
 {'y': 51.3844458, 'x': -0.2926802, 'street_count': 3})

In [34]:
# Look at some nodes
[dx for dx in G_tolworth.neighbors(23780711)], 


all_keys = set()
for _, attrs in G_tolworth.nodes(data=True):
    all_keys.update(attrs.keys())

print(sorted(all_keys))

print(G_tolworth.nodes[start_node])

[dx for dx in G_tolworth.successors(start_node)], [dx for dx in G_tolworth.predecessors(start_node)]

['highway', 'street_count', 'x', 'y']
{'y': 51.381894, 'x': -0.2938243, 'street_count': 3}


([23780702, 23780711, 2578590287], [23780702, 23780711, 2578590287])

In [29]:
## Explore edges
edge1 = G_tolworth.get_edge_data(23779844, 2578590287)#
print(edge1)
edge2 = G_tolworth.get_edge_data(2578590287, 23779844)#
print(edge2)

{0: {'osmid': 251673770, 'highway': 'residential', 'maxspeed': '20 mph', 'name': 'Douglas Road', 'oneway': False, 'reversed': True, 'length': np.float64(294.1048198309258), 'geometry': <LINESTRING (-0.294 51.382, -0.294 51.382, -0.294 51.382, -0.294 51.382, -0....>}}
{0: {'osmid': 251673770, 'highway': 'residential', 'maxspeed': '20 mph', 'name': 'Douglas Road', 'oneway': False, 'reversed': False, 'length': np.float64(294.1048198309258), 'geometry': <LINESTRING (-0.293 51.384, -0.293 51.384, -0.293 51.384, -0.293 51.383, -0....>}}


In [150]:
## Node 1 to node 2
node1_to_node2 = get_turn_node(G_tolworth, 'Douglas Road', 'Ewell Road', 23780711, 'left')

node1_to_node2_nodes = {node: G_tolworth.nodes[node] for node in node1_to_node2}

node1_to_node2_markers = [
    dl.Marker(position=(node1_to_node2_nodes[dx]['y'], node1_to_node2_nodes[dx]['x']), 
              children=dl.Tooltip(f'ID: {dx}, x: {node1_to_node2_nodes[dx]['x']}, y: {node1_to_node2_nodes[dx]['y']}'))
    for dx in node1_to_node2_nodes.keys()
    ]

node1_to_node2_edges = []

## Get the edges
node1_to_node2_edges = [G_tolworth.get_edge_data(node1_to_node2[u], node1_to_node2[u + 1])[0] for u in range(len(node1_to_node2) - 1)]

# for dx in node1_to_node2_edges:
#     if 'geometry' in dx:
#         coords = 

# for u, v, data in G_tolworth.edges(data=True):
#     # Some edges have multiple geometries (from OSM), handle those first
#     if 'geometry' in data:
#         # If geometry is a LineString, extract lat/lon pairs
#         coords = [(point.y, point.x) for point in data['geometry'].coords]
#     else:
#         # Otherwise use straight line between nodes
#         coords = [
#             (G_tolworth.nodes[u]['y'], G_tolworth.nodes[u]['x']),
#             (G_tolworth.nodes[v]['y'], G_tolworth.nodes[v]['x'])
#         ]
    
#     node1_to_node2_edge_lines.append(dl.Polyline(positions=coords, color='blue', weight=2))

node1_to_node2_nodes
pp = dl.Polyline(positions = [(point[1], point[0]) for point in node1_to_node2_edges[0]['geometry'].coords], color='red', weight=2)
pp

Polyline(color='red', positions=[(51.3806375, -0.2944253), (51.3809071, -0.2942964), (51.3815349, -0.2939961), (51.381894, -0.2938243)], weight=2)

In [151]:
## Plot some stuffffffffffffffffffffff
app = JupyterDash(__name__)
app.layout = html.Div([
    dl.Map(center=(centre_lat, centre_lon), zoom=18, children=[
        dl.TileLayer(),
        dl.LayerGroup(node1_to_node2_markers + [pp])
    ], style={'width': '100%', 'height': '4200px'})
])

app.run(mode='inline', port=8051)


/home/dave/anaconda3/envs/mapping_py12/lib/python3.12/site-packages/dash/dash.py:587: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.

